In [70]:
#! pip install pandas google-cloud-bigquery google-auth

In [71]:
#! pip install pandas-gbq

In [72]:
# Importando bibliotecas para o projeto
import pandas as pd
import sqlite3
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [73]:
# Configurando a conexão com o BigQuery usando a chave JSON
key_path = r"C:\Users\marcell.oliveira\Desktop\Chinook\sincere-actor-393814-8209ed1b42a4.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


In [74]:
# Conectar ao banco de dados SQLite
conexao = sqlite3.connect('Chinook_Sqlite.sqlite')

In [75]:
# Obter lista de tabelas no banco de dados
tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conexao)
tabelas 

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
# Nome do dataset no BigQuery
dataset_id = "camada_bronze" 

In [ ]:
# Loop para ler cada tabela e fazer o upload para o BigQuery
for tabela in tabelas['name']:
    print(f"Processando a tabela: {tabela}")
    
    # Ler a tabela do SQLite para um DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {tabela};", conexao)
    
    # Definir o ID da tabela no BigQuery (dataset_id.table_name)
    tabela_id = f"{dataset_id}.{tabela}"
    
    # Configuração da tabela para o BigQuery
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  # Sobrescreve os dados da tabela se já existir
    )
    
    # Fazer o upload do DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df, tabela_id, job_config=job_config)
    job.result()  # Espera até que o job seja concluído
    print(f"Tabela {tabela} carregada com sucesso no BigQuery.")

Processando a tabela: Album
Tabela Album carregada com sucesso no BigQuery.
Processando a tabela: Artist
Tabela Artist carregada com sucesso no BigQuery.
Processando a tabela: Customer
Tabela Customer carregada com sucesso no BigQuery.
Processando a tabela: Employee
Tabela Employee carregada com sucesso no BigQuery.
Processando a tabela: Genre
Tabela Genre carregada com sucesso no BigQuery.
Processando a tabela: Invoice
Tabela Invoice carregada com sucesso no BigQuery.
Processando a tabela: InvoiceLine
Tabela InvoiceLine carregada com sucesso no BigQuery.
Processando a tabela: MediaType
Tabela MediaType carregada com sucesso no BigQuery.
Processando a tabela: Playlist
Tabela Playlist carregada com sucesso no BigQuery.
Processando a tabela: PlaylistTrack
Tabela PlaylistTrack carregada com sucesso no BigQuery.
Processando a tabela: Track
Tabela Track carregada com sucesso no BigQuery.


In [78]:
# Fechar a conexão com o SQLite
conexao.close()